# Coursera Capstone Project - Notebook 
The notebook scrapes the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  to process into a dataframe.  The dataframe is then cleaned and put into a usable form for Week 3. 


In [56]:
# Imports to load the utilities needed to parse and create the data frame 
import xml.etree.ElementTree as ET
import requests
import pandas as pd

In [57]:
# Downloads the Object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
r

<Response [200]>

In [112]:
df = pd.DataFrame([], columns=['Postcode','Borough','Neighbourhood'])

tree = ET.fromstring(r.content)
tables = tree.findall(".//*[@id]/div/table")[0]

rows = tables.findall('.//tbody/tr')
for row in rows: 
    if not 'Not assigned' in str(row[1].text):
        borough = str(row[1].text).replace('\n','')
        if 'None' in borough: 
            borough = row[1][0].text
        
        neighbourhood = str(row[2].text).replace('\n','')
        if 'None' in neighbourhood: 
            neighbourhood = row[2][0].text
        
        if 'Not assigned' in neighbourhood: 
            neighbourhood = borough
        
        df = df.append(
            {'Postcode': str(row[0].text).replace('\n',''), 
             'Borough' : borough, 
             'Neighbourhood' : neighbourhood}, 
            ignore_index=True)

# remove the header data element
df = df.drop(df.index[0])
df.describe()

,Postcode,Borough,Neighbourhood
count,212,212,212
unique,103,11,210
top,M8Y,Etobicoke,St. James Town
freq,8,45,2


In [115]:
df_g = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df_g.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [116]:
df_g[df_g['Borough'].str.contains('Downtown Toronto')]

,Postcode,Borough,Neighbourhood
50,M4W,Downtown Toronto,Rosedale
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town"
52,M4Y,Downtown Toronto,Church and Wellesley
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"
54,M5B,Downtown Toronto,"Ryerson,Garden District"
55,M5C,Downtown Toronto,St. James Town
56,M5E,Downtown Toronto,Berczy Park
57,M5G,Downtown Toronto,Central Bay Street
58,M5H,Downtown Toronto,"Adelaide,King,Richmond"
59,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station"


In [117]:
df_g[df_g['Borough'].str.contains('Queen')]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [121]:
print("The number of rows are ", df_g.shape[0])

The number of rows are  103
